# Running the secondary model

This model was designed to be run after label transfer.  It uses log+1 normalized counts for cells that are labelled as 
neurons (or doublets)

Before running, you need to make sure that you have the packages listed in the imports below.  
Also, I am running tensorflow 2.2.

# step one
### import required python libraries

In [18]:
import tensorflow as tf
import pandas as pd
import numpy as np

import scipy.io
import sklearn.preprocessing
import time
import hashlib

from collections import Counter

### define required functions

In [19]:
## convert the sparse matrix into a log x+1 normalized sparse tensor.
def make_log_maxn_tensor(sparseMatrix,ds_list,ds_type):
  # filter the sparse matrix... (note: cannot be a COO formatted matrix)
  indx = np.equal( np.array(ds_list,dtype=object),np.array(ds_type,dtype=object) )
  x0 = sparseMatrix[indx,:]
  del indx

  # get the log transform...
  x0.data = np.log1p(x0.data)
  # normalize to the max...
  sklearn.preprocessing.normalize(x0, norm="max", axis=1, copy=False)

  # convert the filtered matrix to COO format
  x0 = x0.tocoo()
  indices = np.mat([x0.row, x0.col]).transpose()

  # make a sparse tensor and re-order it...
  x0 = tf.SparseTensor(indices,x0.data,x0.shape)
  x0 = tf.sparse.reorder(x0)
  return x0

Copy the data from github to the VM.  We need to install git-lfs, because the count matrix is too large to be placed in github, so we use the git lfs (large file system. )

In [20]:
! sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [21]:
! rm -rf SeqSeek-Classify-NN
! git lfs clone https://github.com/ArielLevineLabNINDS/SeqSeek-Classify-NN.git
! cd SeqSeek-Classify-NN/

! head SeqSeek-Classify-NN/filtered_neurons_doublets.mtx

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into 'SeqSeek-Classify-NN'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 2), reused 16 (delta 1), pack-reused 0
Unpacking objects: 100% (20/20), done.
Git LFS: (1 of 1 files) 603.08 MB / 603.08 MB
%%MatrixMarket matrix coordinate integer general
28238 25419 47542555
1 1 1
3 1 3
4 1 13
5 1 9
7 1 1
9 1 2
10 1 13
12 1 2


# step two: 
### data loading...

We are going to re-create the test data from the analysis.  In the future, I will create a 
dataset that is only test data, then I will try to create a function that will convert your data
into the appropriat format.  After all, it's more fun to play with your data.  Keep in mind that
this data is mouse spinal cord neurons and doublets only.  

The input data is a sparse matrix created in R from Seurat data.  In Seurat, the count matrices are genes (rows) by cells (columns).  
Tensorflow requires cells (rows) by genes (columns).  I performed the matrix transpose in R, so you won't see it here.

note: it takes just over a minute to read the data on my laptop (macbook pro 2018).

In [22]:
t0 = time.time()
genes = pd.read_csv("SeqSeek-Classify-NN/filtered_neurons_doublets_genes.tsv",names=["gene"])
df = pd.read_csv("SeqSeek-Classify-NN/filtered_neurons_doublets_barcodes.tsv",names=["cell_id"])
lbl = pd.read_csv("SeqSeek-Classify-NN/filtered_neurons_doublets_labels.csv")
t1 = time.time()
print(f"time to read data files: {t1-t0:.3f} s")
print(f"all cell_ids from the labels == cell_ids from matrix: {all(df.cell_id == lbl.cell_id)}")

t0 = time.time()
dirty_neurons = scipy.io.mmread("SeqSeek-Classify-NN/filtered_neurons_doublets.mtx")
t1 = time.time()
print(f"time to read matrix: {t1-t0:.3f} s")
if scipy.sparse.isspmatrix_coo(dirty_neurons):
  print("converting raw counts to CSR")
  t0 = time.time()
  dirty_neurons = dirty_neurons.tocsr()
  t1 = time.time()
  print(f"time to convert matrix: {t1-t0:.3f} s")

time to read data files: 0.071 s
all cell_ids from the labels == cell_ids from matrix: True
time to read matrix: 96.645 s
converting raw counts to CSR
time to convert matrix: 1.004 s


### load the label encoder
The labels must be in the same order as model.

In [11]:
label_encoder = sklearn.preprocessing.LabelEncoder()
label_encoder.classes_ = np.load('SeqSeek-Classify-NN/dirty_neuron_encoder.npy')
ohe_encoder = sklearn.preprocessing.LabelBinarizer()
ohe_encoder.classes_ = np.load('SeqSeek-Classify-NN/dirty_neuron_encoder.npy')

### split the data into train, validation, and test
we are only interested in the test data for this example.  In order to have a consistant train, val, test data split between the data, I take the md5 hash of the barcode, which is a large integer.  I look at the mod 100 of the barcode and take 80% of the data as training data 10% validation, and 10% testing.

In order to use the data, I have to convert the sparse matrix into a sparse tensor.

In [23]:
bchash = list( map(lambda bc: int(hashlib.md5(bc.encode()).hexdigest(),16)%100,lbl.cell_id))
dataset = list( map(lambda hash: "Train" if hash<80 else ("Val" if hash < 90 else "Test"),bchash) )
del bchash
print(dict(Counter(dataset)))

t0 = time.time()
x_test  = make_log_maxn_tensor(dirty_neurons,dataset,"Test")
t1 = time.time()
print(f"time to convert sparse matrix to sparse tensor for the Test data: {t1-t0:.3f} s")

{'Train': 22591, 'Test': 2810, 'Val': 2837}
time to convert sparse matrix to sparse tensor for the Test data: 0.264 s


Normally at prediction time, we would not have labels.  But since we have them, let's use them...

In [24]:
y_test = lbl.final_cluster_assignment.values[ [x == "Test" for x in dataset] ]

we now have the testing data and are ready to run it through the model...

# Step Three:
load the Neural Network

In [25]:
model = tf.keras.models.load_model('SeqSeek-Classify-NN/neurons_doublets.model')
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               6507520   
_________________________________________________________________
cell_type (Dense)            (None, 70)                17990     
Total params: 6,525,510
Trainable params: 6,525,510
Non-trainable params: 0
_________________________________________________________________


In [26]:
pred = model.predict(x_test)
cell_class = np.argmax(pred,axis=1)
cell_type = label_encoder.inverse_transform(cell_class)
called_class = label_encoder.transform(y_test)

df = pd.DataFrame({"predicted_class":cell_class,"called_class":called_class,"predicted_type":cell_type,"called_type":y_test,"probability":np.max(pred,axis=1)} )
df['agree'] = df.predicted_class == df.called_class
df

,predicted_class,called_class,predicted_type,called_type,probability,agree
0,39,39,Garbage,Garbage,0.947911,True
1,39,39,Garbage,Garbage,0.964306,True
2,39,39,Garbage,Garbage,0.097964,True
3,39,39,Garbage,Garbage,0.957435,True
4,1,1,Excit-01,Excit-01,0.538859,True
...,...,...,...,...,...,...
2805,35,35,Excit-35,Excit-35,0.972334,True
2806,62,62,Inhib-23,Inhib-23,0.951047,True
2807,22,22,Excit-22,Excit-22,0.453575,True
2808,35,39,Excit-35,Garbage,0.799530,False
